<a href="https://colab.research.google.com/github/aishwarya2005thakur/Amazon_ML_challenge/blob/main/Dibetes_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install --upgrade pip
!pip install pandas numpy scikit-learn xgboost lightgbm tqdm matplotlib seaborn pillow requests torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
from google.colab import drive
import pandas as pd
import os

# Mount Google Drive
drive.mount('/content/drive')

# === Option 1: Direct CSV Paths (Recommended) ===
# Replace with your actual file paths inside Drive
train_path = "/content/drive/MyDrive/dataset/train.csv"
test_path = "/content/drive/MyDrive/dataset/test.csv"

# === Option 2: If you still have a ZIP file ===
# Uncomment below lines if your dataset is zipped in Drive
# import zipfile
# zip_path = "/content/drive/MyDrive/your_folder/dataset.zip"
# extract_dir = "/content/unzipped_data"
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)
#
# # Find train/test files after unzipping
# for root, _, files in os.walk(extract_dir):
#     for f in files:
#         if f == "train.csv":
#             train_path = os.path.join(root, f)
#         if f == "test.csv":
#             test_path = os.path.join(root, f)

assert os.path.exists(train_path), f"train.csv not found at {train_path}"
assert os.path.exists(test_path), f"test.csv not found at {test_path}"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train['catalog_content'] = train['catalog_content'].fillna('')
test['catalog_content'] = test['catalog_content'].fillna('')

y = train['price'].values
use_log = True

print("Loaded:", train.shape, test.shape)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded: (75000, 4) (75000, 3)


In [ ]:
#feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

train['catalog_content'] = train['catalog_content'].fillna('')
test['catalog_content'] = test['catalog_content'].fillna('')

print("Rebuilding TF-IDF (20k features)...")
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,2), min_df=5)
X_all = pd.concat([train['catalog_content'], test['catalog_content']], axis=0)
tfidf.fit(X_all)
X_train = tfidf.transform(train['catalog_content'])
X_test = tfidf.transform(test['catalog_content'])
print("TF-IDF shapes:", X_train.shape, X_test.shape)

Rebuilding TF-IDF (20k features)...
TF-IDF shapes: (75000, 20000) (75000, 20000)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# -------- Step 3: Extract quantity and unit features --------
import re
from scipy.sparse import hstack
import numpy as np
def extract_pack_qty(text):
    text = str(text).lower()
    patterns = [
        r'pack of (\d+)', r'(\d+)\s*pack', r'(\d+)\s*pcs?', r'(\d+)\s*count',
        r'x\s*(\d+)', r'(\d+)\s*pieces?', r'(\d+)\s*ct\b'
    ]
    for p in patterns:
        m = re.search(p, text)
        if m:
            return float(m.group(1))
    return 1.0

def extract_standard_weight(text):
    text = str(text).lower()
    patterns = [
        (r'([\d]*\.?[\d]+)\s*kg', 1000.0),
        (r'([\d]*\.?[\d]+)\s*g', 1.0),
        (r'([\d]*\.?[\d]+)\s*mg', 0.001),
        (r'([\d]*\.?[\d]+)\s*l', 1000.0),
        (r'([\d]*\.?[\d]+)\s*ml', 1.0),
        (r'([\d]*\.?[\d]+)\s*oz', 28.35),
        (r'([\d]*\.?[\d]+)\s*pound', 453.6),
        (r'([\d]*\.?[\d]+)\s*lb', 453.6),
        (r'([\d]*\.?[\d]+)\s*fl', 29.57)
    ]
    for p, mult in patterns:
        m = re.search(p, text)
        if m:
            try:
                val = float(m.group(1))
                # sanity check: skip nonsense like 0 or extremely small
                if val > 0:
                    return val * mult
            except ValueError:
                continue
    return 1.0

# Apply
train['item_pack_qty'] = train['catalog_content'].apply(extract_pack_qty)
test['item_pack_qty'] = test['catalog_content'].apply(extract_pack_qty)

train['standard_weight_ml'] = train['catalog_content'].apply(extract_standard_weight)
test['standard_weight_ml'] = test['catalog_content'].apply(extract_standard_weight)

# Log-scale
for c in ['item_pack_qty', 'standard_weight_ml']:
    train[c] = np.log1p(train[c])
    test[c] = np.log1p(test[c])

# Combine numeric + TF-IDF
num_feats_train = train[['item_pack_qty', 'standard_weight_ml']].values
num_feats_test  = test[['item_pack_qty', 'standard_weight_ml']].values

X_train_comb = hstack([X_train, num_feats_train])
X_test_comb  = hstack([X_test, num_feats_test])

print(" Combined shape:", X_train_comb.shape, X_test_comb.shape)

 Combined shape: (75000, 20002) (75000, 20002)


In [ ]:
import gc, numpy as np
from sklearn.model_selection import KFold
from scipy.sparse import csr_matrix
import lightgbm as lgb

# Reduce TF-IDF dimensionality (to lighten memory & noise)
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=1e-5)
X_train_sel = sel.fit_transform(X_train)
X_test_sel  = sel.transform(X_test)

# Combine numeric features again
num_feats_train = train[["item_pack_qty", "standard_weight_ml"]].values
num_feats_test  = test[["item_pack_qty", "standard_weight_ml"]].values
X_train_final = csr_matrix(hstack([X_train_sel, num_feats_train]))
X_test_final  = csr_matrix(hstack([X_test_sel,  num_feats_test]))

y = train["price"].values.astype(float)
use_log = True

params = {
    "objective": "regression",
    "metric": "mae",
    "learning_rate": 0.03,      # smaller learning rate
    "num_leaves": 96,           # a bit deeper
    "feature_fraction": 0.75,
    "bagging_fraction": 0.85,
    "bagging_freq": 5,
    "lambda_l2": 1.0,
    "verbosity": -1,
    "seed": 42,
}

NFOLD = 5
kf = KFold(n_splits=NFOLD, shuffle=True, random_state=42)
oof = np.zeros(len(train))
test_preds = np.zeros(len(test))

print("Training tuned LightGBM...")
for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train_final), 1):
    print(f"\n---- Fold {fold}/{NFOLD} ----")
    X_tr, X_val = X_train_final[tr_idx], X_train_final[val_idx]
    y_tr, y_val = y[tr_idx], y[val_idx]
    if use_log:
        y_tr, y_val = np.log1p(y_tr), np.log1p(y_val)
    dtr = lgb.Dataset(X_tr, label=y_tr)
    dval = lgb.Dataset(X_val, label=y_val)
    model = lgb.train(
        params,
        dtr,
        num_boost_round=1500,
        valid_sets=[dval],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50),
            lgb.log_evaluation(200)
        ]
    )
    val_pred  = model.predict(X_val, num_iteration=model.best_iteration)
    test_pred = model.predict(X_test_final, num_iteration=model.best_iteration)
    if use_log:
        val_pred, test_pred = np.expm1(val_pred), np.expm1(test_pred)
    val_pred[val_pred <= 0] = 0.01
    test_pred[test_pred <= 0] = 0.01
    oof[val_idx] = val_pred
    test_preds += test_pred / NFOLD
    del model, dtr, dval, X_tr, X_val, y_tr, y_val
    gc.collect()

smape_val = np.mean(np.abs(oof - y) / ((np.abs(oof) + np.abs(y)) / 2)) * 100
print("\nTuned LightGBM SMAPE:", round(smape_val, 3), "%")

Training tuned LightGBM...

---- Fold 1/5 ----
Training until validation scores don't improve for 50 rounds
[200]	valid_0's l1: 0.535655
[400]	valid_0's l1: 0.518886
[600]	valid_0's l1: 0.511804
[800]	valid_0's l1: 0.508698
[1000]	valid_0's l1: 0.506719
[1200]	valid_0's l1: 0.505256
Early stopping, best iteration is:
[1342]	valid_0's l1: 0.504456

---- Fold 2/5 ----
Training until validation scores don't improve for 50 rounds
[200]	valid_0's l1: 0.526064
[400]	valid_0's l1: 0.510218
[600]	valid_0's l1: 0.503887
[800]	valid_0's l1: 0.500831
[1000]	valid_0's l1: 0.499101
[1200]	valid_0's l1: 0.498284
Early stopping, best iteration is:
[1300]	valid_0's l1: 0.497634

---- Fold 3/5 ----
Training until validation scores don't improve for 50 rounds
[200]	valid_0's l1: 0.528051
[400]	valid_0's l1: 0.512765
[600]	valid_0's l1: 0.506492
[800]	valid_0's l1: 0.503715
[1000]	valid_0's l1: 0.502026
[1200]	valid_0's l1: 0.501165
Early stopping, best iteration is:
[1182]	valid_0's l1: 0.501102

---- F

In [ ]:
import pandas as pd

smape_val = np.mean(np.abs(oof - y) / ((np.abs(oof) + np.abs(y)) / 2)) * 100
print("\nOOF SMAPE:", round(smape_val, 4), "%")

# Save test predictions
submission = pd.DataFrame({
    "sample_id": test["sample_id"],
    "price": test_preds
})
submission.to_csv("test_out_3.csv", index=False)
print(f"Saved test_out_3.csv with {len(submission)} rows")
print(submission.head())


OOF SMAPE: 50.4001 %
Saved test_out_3.csv with 75000 rows
   sample_id      price
0     100179  12.584195
1     245611  19.401508
2     146263  16.220180
3      95658  10.581680
4      36806  39.879938


In [ ]:
import numpy as np
import lightgbm as lgb

def smape_loss(preds, train_data):
    labels = train_data.get_label()
    # Ensure labels and predictions are non-negative before calculating SMAPE
    preds = np.maximum(preds, 0)
    labels = np.maximum(labels, 0)
    denom = (np.abs(labels) + np.abs(preds)) / 2
    # Avoid division by zero for cases where both labels and preds are 0
    denom[denom == 0] = 1e-9
    grad = np.sign(preds - labels) / denom
    hess = 1 / denom
    # Handle potential NaNs or Infs
    grad[np.isnan(grad)] = 0
    hess[np.isnan(hess)] = 0
    grad[np.isinf(grad)] = 0
    hess[np.isinf(hess)] = 0
    return grad, hess

# Keep the existing parameters and disable built-in metric
params["objective"] = "regression"
params["metric"] = "None"

# This cell is now just defining the loss function and updating params.
# The training will happen in the previous cell (nSa-wmMdmQk6)

In [ ]:
# After training
bias = np.median(y / (oof + 1e-6))
oof *= bias
test_preds *= bias


In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(oof.reshape(-1,1), y)
oof = lr.predict(oof.reshape(-1,1))
test_preds = lr.predict(test_preds.reshape(-1,1))


In [ ]:
def extract_brand(text):
    text = str(text).split(' ')[0].lower()
    text = re.sub(r'[^a-z]', '', text)
    return text

train['brand'] = train['catalog_content'].apply(extract_brand)
test['brand'] = test['catalog_content'].apply(extract_brand)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['brand'] = le.fit_transform(train['brand'])
test['brand'] = le.transform(test['brand'])


In [ ]:
for kw in ["sauce","shampoo","bottle","can","snack","soap","cream"]:
    train[f"kw_{kw}"] = train['catalog_content'].str.contains(kw, case=False, regex=False).astype(int)
    test[f"kw_{kw}"] = test['catalog_content'].str.contains(kw, case=False, regex=False).astype(int)


In [ ]:
train['desc_len'] = train['catalog_content'].str.len()
test['desc_len'] = test['catalog_content'].str.len()
train['word_count'] = train['catalog_content'].str.split().apply(len)
test['word_count'] = test['catalog_content'].str.split().apply(len)


In [ ]:
print("\nOOF SMAPE:", round(smape_val, 4), "%")


OOF SMAPE: 50.4001 %


In [ ]:
import pandas as pd, numpy as np, re, gc, os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
from scipy.sparse import hstack, csr_matrix
import lightgbm as lgb

# ===== Fill missing text =====
train['catalog_content'] = train['catalog_content'].fillna('')
test['catalog_content'] = test['catalog_content'].fillna('')

# ===== TF-IDF features =====
print("Building TF-IDF (20k features)...")
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,2), min_df=5)
X_all = pd.concat([train['catalog_content'], test['catalog_content']], axis=0)
tfidf.fit(X_all)
X_train = tfidf.transform(train['catalog_content'])
X_test = tfidf.transform(test['catalog_content'])
print("TF-IDF shapes:", X_train.shape, X_test.shape)

# ===== Custom numeric features =====
def extract_pack_qty(text):
    text = str(text).lower()
    patterns = [
        r'pack of (\d+)', r'(\d+)\s*pack', r'(\d+)\s*pcs?', r'(\d+)\s*count',
        r'x\s*(\d+)', r'(\d+)\s*pieces?', r'(\d+)\s*ct\b'
    ]
    for p in patterns:
        m = re.search(p, text)
        if m:
            return float(m.group(1))
    return 1.0

def extract_standard_weight(text):
    text = str(text).lower()
    patterns = [
        (r'([\d]*\.?[\d]+)\s*kg', 1000.0),
        (r'([\d]*\.?[\d]+)\s*g', 1.0),
        (r'([\d]*\.?[\d]+)\s*mg', 0.001),
        (r'([\d]*\.?[\d]+)\s*l', 1000.0),
        (r'([\d]*\.?[\d]+)\s*ml', 1.0),
        (r'([\d]*\.?[\d]+)\s*oz', 28.35),
        (r'([\d]*\.?[\d]+)\s*pound', 453.6),
        (r'([\d]*\.?[\d]+)\s*lb', 453.6),
        (r'([\d]*\.?[\d]+)\s*fl', 29.57)
    ]
    for p, mult in patterns:
        m = re.search(p, text)
        if m:
            try:
                val = float(m.group(1))
                if val > 0:
                    return val * mult
            except ValueError:
                continue
    return 1.0

# Apply feature extractors
train['item_pack_qty'] = train['catalog_content'].apply(extract_pack_qty)
test['item_pack_qty'] = test['catalog_content'].apply(extract_pack_qty)
train['standard_weight_ml'] = train['catalog_content'].apply(extract_standard_weight)
test['standard_weight_ml'] = test['catalog_content'].apply(extract_standard_weight)

# Log-scale numeric features
for c in ['item_pack_qty', 'standard_weight_ml']:
    train[c] = np.log1p(train[c])
    test[c] = np.log1p(test[c])

# ===== Feature selection to reduce noise =====
sel = VarianceThreshold(threshold=1e-5)
X_train_sel = sel.fit_transform(X_train)
X_test_sel = sel.transform(X_test)

# Combine numeric features
num_feats_train = train[['item_pack_qty', 'standard_weight_ml']].values
num_feats_test  = test[['item_pack_qty', 'standard_weight_ml']].values
X_train_final = csr_matrix(hstack([X_train_sel, num_feats_train]))
X_test_final  = csr_matrix(hstack([X_test_sel,  num_feats_test]))

print("Final shape:", X_train_final.shape, X_test_final.shape)

# ===== LightGBM with DART boosting =====
y = train["price"].values.astype(float)
NFOLD = 5
kf = KFold(n_splits=NFOLD, shuffle=True, random_state=42)
oof = np.zeros(len(y))
test_preds = np.zeros(len(test))

params = {
    "objective": "regression",
    "metric": "mae",  # we'll compute SMAPE manually
    "boosting_type": "dart",
    "learning_rate": 0.03,
    "num_leaves": 64,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 4,
    "lambda_l2": 4,
    "max_depth": -1,
    "verbosity": -1,
    "n_jobs": -1,
    "seed": 42,
}

print("Training DART LightGBM (SMAPE objective)...")

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train_final, y)):
    print(f"\n---- Fold {fold+1}/{NFOLD} ----")
    X_tr, X_val = X_train_final[tr_idx], X_train_final[val_idx]
    y_tr, y_val = y[tr_idx], y[val_idx]

    # Log-transform prices
    y_tr_log = np.log1p(y_tr)
    y_val_log = np.log1p(y_val)

    dtr = lgb.Dataset(X_tr, label=y_tr_log)
    dval = lgb.Dataset(X_val, label=y_val_log)

    model = lgb.train(
        params,
        dtr,
        valid_sets=[dval],
        num_boost_round=3000,
        callbacks=[lgb.log_evaluation(200)]
    )

    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    test_pred = model.predict(X_test_final, num_iteration=model.best_iteration)

    # Inverse log transform
    val_pred = np.expm1(val_pred)
    test_pred = np.expm1(test_pred)

    oof[val_idx] = val_pred
    test_preds += test_pred / NFOLD


    del model, dtr, dval, X_tr, X_val, y_tr, y_val
    gc.collect()

# ===== SMAPE calculation =====
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8))

oof_smape = smape(y, oof)
print(f"\nOOF SMAPE: {oof_smape:.4f}%")

# ===== Save test predictions =====
submission = pd.DataFrame({
    "sample_id": test["sample_id"],
    "price": test_preds
})
submission.to_csv("test_out_4.csv", index=False)
print(f"Saved test_out_4.csv with {len(submission)} rows")


🧠 Building TF-IDF (20k features)...
TF-IDF shapes: (75000, 20000) (75000, 20000)
Final shape: (75000, 15281) (75000, 15281)
Training DART LightGBM (SMAPE objective)...

---- Fold 1/5 ----
[200]	valid_0's l1: 0.892413
[400]	valid_0's l1: 0.67469
[600]	valid_0's l1: 0.605533
[800]	valid_0's l1: 0.557927
[1000]	valid_0's l1: 0.547914
[1200]	valid_0's l1: 0.539444
[1400]	valid_0's l1: 0.530034
[1600]	valid_0's l1: 0.523665
[1800]	valid_0's l1: 0.524322
[2000]	valid_0's l1: 0.518801
[2200]	valid_0's l1: 0.517873
[2400]	valid_0's l1: 0.515487
[2600]	valid_0's l1: 0.514074
[2800]	valid_0's l1: 0.513196
[3000]	valid_0's l1: 0.510344

---- Fold 2/5 ----
[200]	valid_0's l1: 0.896355
[400]	valid_0's l1: 0.672646
[600]	valid_0's l1: 0.600237
[800]	valid_0's l1: 0.550454
